In [ ]:
!pip install popgym # base environments only, only requires numpy and gymnasium
!pip install --use-pep517 "popgym[navigation]" # also include navigation environments, which require mazelib
!pip install "popgym[baselines]" # environments and memory baselines

In [ ]:
import popgym
from popgym.wrappers import PreviousAction, Antialias, Markovian, Flatten, DiscreteAction
from popgym.core.observability import Observability, STATE
env_classes = popgym.envs.ALL.keys()
set_of_action_space=set()
for num,i in enumerate(env_classes):
  env=i()
  if str(env.action_space)=="Discrete(4)":
    print("Class #",num,i)
    print("Number of Actions : ",env.action_space)

  set_of_action_space.add(str(env.action_space))



In [ ]:
nums_of_action_state_types=dict()
for num,i in enumerate(set_of_action_space):
  print(num)
  count=0
  for j in env_classes:
    env=j()
    x=str(env.action_space)
    if x==i:
      count+=1
  print(i,"count:",count)
  nums_of_action_state_types[i]=count


nums_of_action_state_types=(dict(sorted(nums_of_action_state_types.items(), key=lambda x:x[1],reverse=True)))


0
Discrete(52) count: 2
1
Discrete(16) count: 1
2
Discrete(30) count: 1
3
MultiDiscrete([12 12]) count: 1
4
MultiDiscrete([8 8]) count: 2
5
Discrete(26) count: 2
6
MultiDiscrete([4 4]) count: 1
7
Box(-2.0, 2.0, (1,), float32) count: 6
8
Discrete(10) count: 1
9
MultiDiscrete([6 6]) count: 1
10
Discrete(2) count: 12
11
Discrete(20) count: 1
12
Discrete(104) count: 1
13
Discrete(4) count: 15
14
MultiDiscrete([10 10]) count: 1


In [ ]:
import re

for x in nums_of_action_state_types.keys():
  tasks = set()
  env_classes = popgym.envs.ALL.keys()
  set_of_action_space=set()
  print(x)
  count=0
  for i in (env_classes):
    env=i()

    if str(env.action_space)==x:



      if re.split(r"Easy|Medium|Hard",(str(i)[20:-2]).split(".")[1])[0] not in tasks:
        count+=1
        print("task: #",count,re.split(r"Easy|Medium|Hard",(str(i)[20:-2]).split(".")[1])[0])
      tasks.add(str(re.split(r"Easy|Medium|Hard",(str(i)[20:-2]).split(".")[1])[0]))



Discrete(4)
task: # 1 Autoencode
task: # 2 RepeatPrevious
task: # 3 RepeatFirst
task: # 4 LabyrinthExplore
task: # 5 LabyrinthEscape
Discrete(2)
task: # 1 PositionOnlyCartPole
task: # 2 VelocityOnlyCartPole
task: # 3 NoisyPositionOnlyCartPole
task: # 4 HigherLower
Box(-2.0, 2.0, (1,), float32)
task: # 1 PositionOnlyPendulum
task: # 2 NoisyPositionOnlyPendulum
Discrete(52)
task: # 1 Concentration
MultiDiscrete([8 8])
task: # 1 Battleship
task: # 2 MineSweeper
Discrete(26)
task: # 1 CountRecall
Discrete(16)
task: # 1 CountRecall
Discrete(30)
task: # 1 MultiarmedBandit
MultiDiscrete([12 12])
task: # 1 Battleship
MultiDiscrete([4 4])
task: # 1 MineSweeper
Discrete(10)
task: # 1 MultiarmedBandit
MultiDiscrete([6 6])
task: # 1 MineSweeper
Discrete(20)
task: # 1 MultiarmedBandit
Discrete(104)
task: # 1 Concentration
MultiDiscrete([10 10])
task: # 1 Battleship


In [ ]:
env = popgym.envs.labyrinth_explore.LabyrinthExploreEasy()
print(env.action_space)

Discrete(4)


In [ ]:
env = popgym.envs.position_only_cartpole.PositionOnlyCartPoleEasy()
wrapped_env = PreviousAction(env)
print(wrapped_env.action_space)
wrapped_env = Antialias(wrapped_env)
DiscreteAction(Flatten(wrapped_env))
print(env)

Discrete(2)
<PositionOnlyCartPoleEasy instance>


In [ ]:
wrapped_env = Markovian(wrapped_env, Observability.FULL_IN_INFO_DICT)
wrapped_env.reset()
obs, reward, terminated, truncated, info = wrapped_env.step(wrapped_env.action_space.sample())
print(obs)
print(info[STATE])
print(wrapped_env.action_space)
wrapped_env = Flatten(wrapped_env)
print(wrapped_env.action_space.n)

(array([ 0.03581224, -0.03101058], dtype=float32), 1, 0)
[ 0.03581224  0.19563742 -0.03101058 -0.2554924 ]
Discrete(2)
2


In [ ]:
# import popgym
# import ray
# from torch import nn
# from popgym.baselines.ray_models.ray_gru import GRU
# # See what GRU-specific hyperparameters we can set
# print(GRU.MODEL_CONFIG)
# # Show other settable model hyperparameters like
# # what the actor/critic branches look like,
# # what hidden size to use,
# # whether to add a positional embedding, etc.
# print(GRU.BASE_CONFIG)
# # How long the temporal window for backprop is
# # This doesn't need to be longer than 1024
# bptt_size = 1024
# config = {
# "model": {
#     "max_seq_len": bptt_size,
#     "custom_model": GRU,
#     "custom_model_config": {
#     # Override the hidden_size from BASE_CONFIG
#     # The input and output sizes of the MLP feeding the memory model
#     "preprocessor_input_size": 128,
#     "preprocessor_output_size": 64,
#     "preprocessor": nn.Sequential(nn.Linear(128, 64), nn.ReLU()),
#     # this is the size of the recurrent state in most cases
#     "hidden_size": 128,
#     # We should also change other parts of the architecture to use
#     # this new hidden size
#     # For the GRU, the output is of size hidden_size
#     "postprocessor": nn.Sequential(nn.Linear(128, 64), nn.ReLU()),
#     "postprocessor_output_size": 64,
#     # Actor and critic networks
#     "actor": nn.Linear(64, 64),
#     "critic": nn.Linear(64, 64),
#     # We can also override GRU-specific hyperparams
#     "num_recurrent_layers": 1,
#     "resources_per_trial":2
#     },
# },
# # Some other rllib defaults you might want to change
# # See https://docs.ray.io/en/latest/rllib/rllib-training.html#common-parameters
# # for a full list of rllib settings
# #
# # These should be a factor of bptt_size
# "sgd_minibatch_size": bptt_size * 4,
# # Should be a factor of sgd_minibatch_size
# "train_batch_size": bptt_size * 8,
# # The ENVIRONTMENT USED we are training on
# "env": "popgym-ConcentrationEasy-v0",
# # You probably don't want to change these values
# "rollout_fragment_length": bptt_size,
# "framework": "torch",
# "horizon": bptt_size,
# "batch_mode": "complete_episodes",
# }
# # Stop after 50k environment steps
# ray.tune.run("PPO", config=config, stop={"timesteps_total": 50_000})

# Finding the State Space of all environments with Discrete(4) action space

In [ ]:
envs=[popgym.envs.autoencode.Autoencode,
      popgym.envs.labyrinth_escape.LabyrinthEscape,
      popgym.envs.labyrinth_explore.LabyrinthExplore,
      popgym.envs.repeat_previous.RepeatPrevious,
      popgym.envs.repeat_first.RepeatFirst]
for i in envs:
  env=i()
  print("Name of the ENV :",str(env)[1:-1])
  print("Obs space : ",env.observation_space)
  #print("Obeservabiity level of the agent: ",env.observability_level) #all are partial obs
  print("Full state space :",env.state_space)

  print("State space shape :" ,env.state_space.shape)
  #print("Reward Range: ",env.reward_range) #all ranges from -inf to inf

  print("\n \n")



Name of the ENV : Autoencode instance
Obs space :  Tuple(Discrete(2), Discrete(4))
Full state space : Tuple(MultiDiscrete([4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]), Discrete(2), Box(0.0, 1.0, (1,), float32))
State space shape : None

 

Name of the ENV : LabyrinthEscape instance
Obs space :  MultiDiscrete([5 5 5 5 5 5 5 5 5 5])
Full state space : MultiDiscrete([[5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]])
State space shape : (11, 11)

 

Name of the ENV : LabyrinthExplore instance
Obs space :  MultiDiscrete([5 5 5 5 5 5 5 5 5 5])
Full state space : MultiDiscrete([[5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5 5 5 5 5 5 5 5]
 [5 5 5 5

In [ ]:
envs=env_classes
for i in envs:
  env=i()
  print("Name of the ENV :",str(env)[1:-1])
  print("Action Space : ",env.action_space)
  print("Obs space : ",env.observation_space)
  print("Full state space :",env.state_space,"\n \n")

Name of the ENV : AutoencodeEasy instance
Action Space :  Discrete(4)
Obs space :  Tuple(Discrete(2), Discrete(4))
Full state space : Tuple(MultiDiscrete([4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]), Discrete(2), Box(0.0, 1.0, (1,), float32)) 
 

Name of the ENV : RepeatPreviousEasy instance
Action Space :  Discrete(4)
Obs space :  Discrete(4)
Full state space : Tuple(MultiDiscrete([4 4 4 4]), Box(0.0, 1.0, (4,), float32)) 
 

Name of the ENV : RepeatFirstEasy instance
Action Space :  Discrete(4)
Obs space :  Discrete(4)
Full state space : Tuple(Discrete(4), Box(0.0, 1.0, (4,), float32)) 
 

Name of the ENV : CountRecallEasy instance
Action Space :  Discrete(26)
Obs space :  MultiDiscrete([2 2])
Full state space : Tuple(Box(0.0, 1.0, (2,), float32), Box(0.0, 1.0, (2,), float32), MultiDiscrete([2 2])) 
 

Name of the ENV : PositionOnlyCartPoleEasy instance
Action Space :  Discrete(2)
Obs space :  Box([-4.8        -0.41887903

# Trying to render the maze environment

In [ ]:
import pygame

from popgym.envs.labyrinth_escape import LabyrinthEscape

if __name__ == "__main__":
    e = LabyrinthEscape(maze_dims=(12, 12), episode_length=1024)
    obs = e.reset(seed=23)
    e.render()

    # pygame.init()
    # while not done:
    #     events = pygame.event.get()
    #     action = None
    #     for event in events:
    #         if event.type == pygame.KEYDOWN:
    #             if event.key == pygame.K_LEFT:
    #                 action = 0
    #             if event.key == pygame.K_RIGHT:
    #                 action = 1
    #             if event.key == pygame.K_UP:
    #                 action = 2
    #             if event.key == pygame.K_DOWN:
    #                 action = 3
    #     if action is not None:
    #         obs, reward, done, info = e.step(action)
    #         e.render()
    #         print(reward)

██████🛑████████████████
██                      ██
██  ██████████████████  ██
██              ██      ██
██████████████  ██  ██████
██  ██      ██  ██      ██
██  ██  ██  ██  ██████  ██
██      ██      ██  ██  ██
██████████████████  ██  ██
██  ██              ██  ██
██  ██  ██████████████  ██
██                      ██
██████████████🚎██████████


## Labrinth Explore


In [ ]:
import pygame

from popgym.envs.labyrinth_explore import LabyrinthExplore

if __name__ == "__main__":
    e = LabyrinthExplore(maze_dims=(12, 12), episode_length=1024)
    obs = e.reset(seed=23)
    e.render()

██████████████████████████
██                      ██
██  ██████████████████  ██
██              ██      ██
██████████████  ██  ██████
██  ██      ██  ██      ██
██  ██  ██  ██  ██████  ██
██      ██      ██  ██  ██
██████████████████  ██  ██
██  ██              ██  ██
██  ██  ██████████████  ██
██                      ██
██████████████🚎██████████


In [ ]:
if __name__ == "__main__":
    e = LabyrinthEscape(maze_dims=(10,10), episode_length=1024)

    obs = e.reset()
    e.render()
    print("\n")
    obs=e.reset()
    e.render()
    print("\n")


██████████████🛑████
██                  ██
██████████████  ██  ██
██      ██  ██  ██  ██
██  ██  ██  ██  ██  ██
██  ██      ██  ██  ██
██  ██████  ██████  ██
██      ██          ██
██████████████████  ██
██                  ██
██████████████████🚎██


██████████████████████
██      ██          ██
██  ██  ██████████  ██
🛑██      ██      ██
██  ██████  ██  ██  ██
██      ██  ██  ██  🚎
██████  ██  ██  ██  ██
██      ██      ██  ██
██  ██████████████  ██
██              ██  ██
██████████████████████




In [ ]:
from popgym import envs

In [ ]:
if __name__ == "__main__":
    e = LabyrinthExplore(maze_dims=(12, 12), episode_length=1024)

    obs = e.reset()
    e.render()
    print("\n")
    obs=e.reset()
    e.render()
    print("\n")


██🚎██████████████████████
██      ██      ██      ██
██  ██  ██████  ██████  ██
██  ██      ██      ██  ██
██  ██████  ██  ██  ██  ██
██  ██      ██  ██      ██
██████  ██████  ██████  ██
██              ██      ██
██████████  ██████  ██████
██      ██      ██  ██  ██
██  ██████  ██  ██  ██  ██
██          ██  ██      ██
██████████████████████████


██████████🚎██████████████
██                      ██
██████  ██  ██████████████
██      ██      ██      ██
██  ██████████  ██  ██  ██
██  ██      ██      ██  ██
██  ██████  ██████████  ██
██      ██  ██      ██  ██
██████████  ██  ██  ██  ██
██          ██  ██  ██  ██
██  ██████████  ██  ██  ██
██              ██      ██
██████████████████████████




In [ ]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
from popgym.envs.labyrinth_escape import LabyrinthEscapeEasy
import gymnasium as gym
import popgym  # noqa: F401


if __name__ == "__main__":
    e = gym.make("popgym-LabyrinthEscapeEasy-v0")
    obs = e.reset(seed=23)
    img = e.render()
    env.close()


██████🚎██████████████
██                  ██
██  ██████████████  ██
██              ██  ██
██████████████  ██  ██
██  ██      ██  ██  ██
██  ██  ██  ██  ██  ██
██      ██      ██  ██
██████████████████  ██
██                  ██
██████████🛑████████


In [ ]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
from popgym.envs.labyrinth_escape import LabyrinthEscapeEasy
import gymnasium as gym
import popgym  # noqa: F401


if __name__ == "__main__":
    e = gym.make("popgym-LabyrinthEscapeMedium-v0")
    obs = e.reset(seed=23)
    img = e.render()
    env.close()


██████████████████████🛑████
██      ██          ██      ██
██  ██████  ██████  ██  ██  ██
██      ██  ██      ██  ██  ██
██████  ██  ██  ██████████  ██
██  ██      ██      ██      ██
██  ██  ██████████  ██  ██████
██      ██          ██      ██
██  ██████████████  ██████  ██
██              ██  ██      ██
██████████████  ██████  ██████
██              ██          ██
██  ██  ██████████  ██████  ██
██  ██              ██      ██
██🚎██████████████████████████


In [ ]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
from popgym.envs.labyrinth_escape import LabyrinthEscapeEasy
import gymnasium as gym
import popgym  # noqa: F401


if __name__ == "__main__":
    e = gym.make("popgym-LabyrinthEscapeHard-v0")
    obs = e.reset(seed=23)
    img = e.render()
    env.close()


██████████████████████████████████████
██          ██          ██          ██
██████  ██████  ██  ██  ██  ██████  ██
██      ██      ██  ██      ██      ██
██  ██████  ██████  ██████  ██████  ██
🛑        ██      ██  ██      ██  ██
██████████████████  ██  ██████  ██  ██
██                  ██      ██  ██  🚎
██  ██████████████████  ██████  ██  ██
██  ██  ██      ██  ██      ██  ██  ██
██  ██  ██  ██  ██  ██████  ██  ██  ██
██      ██  ██      ██      ██  ██  ██
██  ██████  ██████  ██  ██  ██  ██████
██      ██  ██  ██  ██  ██          ██
██████  ██  ██  ██  ██████  ██████  ██
██      ██      ██      ██  ██      ██
██  ██████████████████  ██  ██  ██████
██                      ██  ██      ██
██████████████████████████████████████


In [ ]:

if __name__ == "__main__":
    e = gym.make("popgym-LabyrinthExploreHard-v0")
    obs = e.reset(seed=23)
    img = e.render()
    env.close()


██████████████████████████████████████
██          ██          ██          ██
██████  ██████  ██  ██  ██  ██████  ██
██      ██      ██  ██      ██      ██
██  ██████  ██████  ██████  ██████  ██
██          ██      ██  ██      ██  ██
██████████████████  ██  ██████  ██  ██
██                  ██      ██  ██  🚎
██  ██████████████████  ██████  ██  ██
██  ██  ██      ██  ██      ██  ██  ██
██  ██  ██  ██  ██  ██████  ██  ██  ██
██      ██  ██      ██      ██  ██  ██
██  ██████  ██████  ██  ██  ██  ██████
██      ██  ██  ██  ██  ██          ██
██████  ██  ██  ██  ██████  ██████  ██
██      ██      ██      ██  ██      ██
██  ██████████████████  ██  ██  ██████
██                      ██  ██      ██
██████████████████████████████████████


In [ ]:
import gymnasium as gym
import popgym  # noqa: F401
from popgym.envs.labyrinth_escape import LabyrinthEscapeHard
# Create the environment
env = gym.make('popgym-LabyrinthEscapeHard-v0')

# Get the size of the state space
state_space_size = env.observation_space.shape[0] # Assuming it's a Box space
print("State Space Size:", state_space_size)

State Space Size: 10


In [ ]:
env.observation_space

MultiDiscrete([5 5 5 5 5 5 5 5 5 5])

n Gym, a multi-discrete action space refers to an action space where there are multiple discrete dimensions, each with its own set of discrete actions. This is in contrast to a standard discrete action space where there is a single discrete action that the agent can choose from.

For example, consider a robotic arm with three joints. Each joint can be moved independently, and the action space for controlling the robotic arm might look like this:

python
Copy code
import gym

# Create a multi-discrete action space environment
env = gym.make('YourMultiDiscreteEnvironmentHere')

# Get the action space
action_space = env.action_space

# Print the action space
print("Action Space:", action_space)
In this case, the action_space might be a MultiDiscrete space with three dimensions, each representing the possible actions for the corresponding joint.

python
Copy code
MultiDiscrete([num_actions_joint_1, num_actions_joint_2, num_actions_joint_3])
Here, num_actions_joint_1, num_actions_joint_2, and num_actions_joint_3 represent the number of discrete actions available for each joint.

For example, if num_actions_joint_1 = 3, num_actions_joint_2 = 5, and num_actions_joint_3 = 2, the agent can choose from 3 actions for the first joint, 5 actions for the second joint, and 2 actions for the third joint in each step.

Agents interacting with environments with multi-discrete action spaces need to provide a list of integers as actions, one for each dimension of the multi-discrete space. Each integer corresponds to the index of the action to be taken in the corresponding dimension.

Keep in mind that the specifics of multi-discrete action spaces can vary depending on the environment, so it's important to refer to the documentation of the specific Gym environment you are working with for detailed information.




User
what is multi discrete  observation state space in gymnasium RL
ChatGPT
In Gym, a multi-discrete observation space refers to an observation space where there are multiple discrete dimensions, each representing a different aspect of the environment's state. This is in contrast to a standard discrete or continuous observation space where there is a single dimension or set of dimensions describing the state.

Here's an example of how you might work with a multi-discrete observation space in Gym:

python
Copy code
import gym

# Create a multi-discrete observation space environment
env = gym.make('YourMultiDiscreteEnvironmentHere')

# Get the observation space
observation_space = env.observation_space

# Print the observation space
print("Observation Space:", observation_space)
In this case, the observation_space might be a MultiDiscrete space with several dimensions, each representing a different aspect of the environment's state. The MultiDiscrete space is defined by a list of integers, where each integer represents the number of possible values for that dimension.

python
Copy code
MultiDiscrete([num_values_dim_1, num_values_dim_2, ..., num_values_dim_n])
Here, num_values_dim_1, num_values_dim_2, etc., represent the number of possible values for each dimension of the observation space.

Agents interacting with environments with multi-discrete observation spaces will receive an observation, which is a list of integers, where each integer corresponds to the value in the corresponding dimension of the multi-discrete space.

As with action spaces, it's important to refer to the documentation of the specific Gym environment you are working with to understand the structure and meaning of the different dimensions in the multi-discrete observation space. The specifics can vary depending on the environment.

# Modifying small environment to become same size as largest env

In [ ]:
import pygame

from popgym.envs.labyrinth_escape import LabyrinthEscape

if __name__ == "__main__":
    e = LabyrinthEscape(maze_dims=(10, 10), episode_length=1024)
    obs = e.reset(seed=23)
    #e.render()


In [ ]:
import gymnasium as gym
from gymnasium.wrappers import PixelObservationWrapper,ResizeObservation,TransformObservation
x=gym.Wrapper(e)
print(x)
print(x.action_space)
print(x.observation_space, x.reward_range, x.metadata )


<Wrapper<LabyrinthEscape instance>>
Discrete(4)
MultiDiscrete([5 5 5 5 5 5 5 5 5 5]) (-inf, inf) {'render_modes': []}


In [ ]:

# Replace "popgym-LabyrinthEscapeEasy-v0" with the actual environment name
env = gym.make("popgym-LabyrinthEscapeEasy-v0")

# Get the state space
state_space = env.observation_space

# Assuming it's a MultiDiscrete space
assert isinstance(state_space, gym.spaces.MultiDiscrete)

# Get the number of discrete values for each dimension
num_values_per_dimension = state_space.nvec
print(num_values_per_dimension)


# Remember to close the environment when done
env.close()



[5 5 5 5 5 5 5 5 5 5]


In [ ]:
from gymnasium.spaces import MultiDiscrete
import popgym
from popgym.envs.labyrinth_escape import LabyrinthEscapeEasy

e_small = LabyrinthEscapeEasy()
obs = e_small.reset(seed=24)

print(type(e_small.state_space))
# e_small.state_space=MultiDiscrete(np.array([[5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
#  [5, 5 ,5 ,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]))
print(e_small.state_space.shape)
print(e_small.state_space.sample())
e_small.render()

<class 'gymnasium.spaces.multi_discrete.MultiDiscrete'>
(11, 11)
[[0 3 4 2 1 3 4 4 3 1 0]
 [4 0 4 1 4 0 2 0 4 1 1]
 [1 0 4 3 2 2 4 4 3 1 4]
 [1 4 2 4 1 3 2 0 0 2 3]
 [2 2 0 4 1 2 3 3 4 0 0]
 [2 3 3 3 1 0 4 3 3 4 0]
 [1 1 0 3 3 4 3 0 0 4 0]
 [1 2 2 2 3 0 3 1 0 0 2]
 [3 0 3 4 4 4 1 3 4 4 4]
 [0 2 0 2 4 3 0 4 2 3 0]
 [4 0 3 1 1 3 4 2 4 2 3]]
██████████████████████
██                  ██
██████  ██  ██████  ██
🚎      ██  ██      ██
██  ██████  ██  ██████
██      ██  ██      🛑
██████  ██  ██████  ██
██      ██      ██  ██
██  ██████████████████
██                  ██
██████████████████████


In [ ]:
import popgym
from popgym.envs.labyrinth_escape import LabyrinthEscapeHard

e_large = LabyrinthEscapeHard()
obs = e_large.reset(seed=23)

print(type(e_large.state_space))
print(e_large.state_space.shape)
print(e_large.state_space.sample())
e.render()

<class 'gymnasium.spaces.multi_discrete.MultiDiscrete'>
(19, 19)
[[3 3 4 4 0 2 4 1 2 1 1 1 2 4 2 2 1 0 0]
 [0 2 4 1 2 1 4 2 4 2 2 1 0 3 0 2 2 1 2]
 [2 0 0 2 0 1 2 0 1 4 2 0 2 1 0 3 4 2 1]
 [4 0 4 4 3 0 2 4 3 3 4 1 1 3 3 3 1 2 0]
 [1 4 2 0 2 2 0 0 0 3 0 3 2 2 2 4 4 3 1]
 [2 1 2 0 4 1 3 0 0 4 2 4 0 1 4 0 1 3 2]
 [3 2 1 2 0 4 2 2 3 4 0 3 2 4 3 4 1 0 0]
 [4 2 1 3 0 4 4 1 4 3 1 2 2 4 0 2 4 4 4]
 [4 4 3 0 4 2 0 0 3 1 3 3 0 2 2 2 3 4 0]
 [3 1 0 2 3 1 1 1 1 2 4 1 2 2 0 0 2 2 2]
 [4 0 3 4 0 4 1 4 2 1 1 2 2 4 0 4 0 3 4]
 [2 3 1 4 0 4 0 3 1 4 0 2 0 0 3 3 1 4 2]
 [4 2 2 3 2 3 0 0 0 1 2 4 4 0 3 2 0 2 1]
 [4 0 3 3 3 1 4 0 4 0 0 3 0 0 4 4 2 2 0]
 [1 0 1 3 3 1 4 3 2 1 2 2 0 1 0 4 4 4 0]
 [0 1 3 1 4 3 2 3 3 4 0 0 0 4 2 3 2 0 2]
 [4 2 3 1 1 2 1 4 0 4 4 3 2 3 1 2 3 2 0]
 [1 1 4 0 4 1 3 2 4 4 2 4 4 0 4 1 4 4 1]
 [4 1 3 1 0 0 2 4 0 2 0 0 4 0 2 0 2 1 1]]
██████████████████████████████████████
██          ██          ██          ██
██████  ██████  ██  ██  ██  ██████  ██
██      ██      ██  ██      ██      ██

array([[0, 1],
       [1, 0]])